In [3]:
import pandas as pd
import os
import pyarrow.parquet as pq
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.models import resnet50, resnet18

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip ./drive/MyDrive/icecube/ice_cube_small.zip

Archive:  ./drive/MyDrive/icecube/ice_cube_small.zip
   creating: ice_cube_small/
  inflating: ice_cube_small/train_meta.parquet  
  inflating: ice_cube_small/test_meta.parquet  
  inflating: ice_cube_small/sensor_geometry.csv  
  inflating: ice_cube_small/sample_submission.parquet  
   creating: ice_cube_small/train_2/
  inflating: ice_cube_small/train_2/batch_3.parquet  
  inflating: ice_cube_small/train_2/batch_2.parquet  
  inflating: ice_cube_small/train_2/batch_1.parquet  
   creating: ice_cube_small/test/
  inflating: ice_cube_small/test/batch_661.parquet  


In [7]:
!cp ./drive/MyDrive/model_101.pth .

In [8]:
!git clone https://github.com/WaShindeiru/IceCubeNeutrino

Cloning into 'IceCubeNeutrino'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 44 (delta 20), reused 14 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (44/44), 384.59 KiB | 1.56 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [9]:
!cp ./IceCubeNeutrino/* .

In [ ]:
# def get_batch(batchfile):
#     batch1 = pq.ParquetFile(batchfile)
#     it = batch1.iter_batches()
#     batch1 = next(it).to_pandas()
#     temp = batchfile.split('/')[-1].split('batch_')[1].split('.')[0]
#     batch1['Batch'] = temp
#     return(batch1)
#
# def get_pq(pqfile):
#     pq_df = pq.ParquetFile(pqfile)
#     it = pq_df.iter_batches()
#     pq_df = next(it).to_pandas()
#     return(pq_df)

In [10]:
path = "./ice_cube_small"
sensor = pd.read_csv(path + '/sensor_geometry.csv')
train_meta = pq.ParquetFile(path + '/train_meta.parquet')
# # train_meta = pq.ParquetFile("/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train/batch_1.parquet")
# it = train_meta.iter_batches()
# train_meta = next(it).to_pandas()

In [ ]:
# train_meta.head(20)

In [ ]:
# i = 1
# while True:
#     try:
#         item = next(it)
#         i += 1
#     except StopIteration:
#         break
#
# print(i)

In [ ]:
# print(train_meta)

In [ ]:
# path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train/'
# batchfile = path_batch + 'batch_1.parquet'
# batch1 = pq.ParquetFile(batchfile)
# it = batch1.iter_batches()
# batch1 = next(it).to_pandas()
# batch1.head(100)

In [ ]:
# path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train/'
# sensor_info = [get_batch(path_batch+'batch_' + str(i+1) + '.parquet') for i in tqdm(range(2))]
# sensor_info_df = pd.concat(sensor_info).reset_index()

In [ ]:
# sensor_info_df.head()

In [ ]:
# plt.figure(figsize = (20,10))
# sns.set(font_scale = 1,style = 'ticks')
# for i in range(16):
#     plt.subplot(4,4,i+1)
#     df = sensor_info_df[(sensor_info_df.event_id==sensor_info_df.event_id.unique()[i])]
#     sns.lineplot(data = df,x = 'time',y = 'charge',hue = 'auxiliary',)
#     plt.title('Batch 1, Event ID'+str(sensor_info_df.event_id.unique()[i]))
#     plt.legend(loc = 'upper right')
#     sns.despine()
# plt.tight_layout()

In [11]:
from Resnet import resnet101
import torch.nn as nn
model = resnet101(pretrained = False)
model.fc = nn.Sequential(nn.ReLU(),nn.Linear(in_features=2048, out_features=2)) # Changed FC layer for our task


In [12]:
model.load_state_dict(torch.load("./model_101.pth", weights_only=True))

<All keys matched successfully>

In [ ]:
# import torch.nn as nn
# model = resnet34(pretrained = False)
# model.fc = nn.Sequential(nn.ReLU(),nn.Linear(in_features=512, out_features=2)) # Changed FC layer for our task
# # model = model.to(device)

In [ ]:
# from CustomConvolution import CustomConvolution
# model = CustomConvolution()
# # model = model.to(device)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

def evaluation(dataloader):
    predictions = torch.tensor([], dtype=torch.float).to(device) # Tensor for prediction value appending
    actual = torch.tensor([], dtype=torch.float).to(device) # Tensor for answer value appending
    with torch.no_grad():
        model.eval()
        for data in dataloader:
            inputs, values = data['input_tensor'].float().to(device),data['label'].to(device)
            outputs = model(inputs).to(device)
            predictions = torch.cat((predictions, torch.stack([torch.argmax(o) for o in outputs])),0)
            actual = torch.cat((actual, values), 0)
    predictions = predictions.cpu().numpy()
    actual = actual.cpu().numpy()
    rmse = np.sqrt(mean_squared_error(predictions, actual))
    return rmse

In [ ]:
from torch import optim
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'


pqfile = path + '/train_meta.parquet'
path_batch = path + '/train_2/'

batch_num=2 # There are 660 batches total, and the batch number should be iterated in range(660).
lr = 1e-06
num_epochs = 1
batch_size = 4

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
# loss_function = nn.BCEWithLogitsLoss().to(device)
loss_function = nn.MSELoss().to(device)

In [ ]:
from torch.utils.data import DataLoader
from Icecube_Dataloader import IceCube_Dataloader, collate_fn

ice_dataset = IceCube_Dataloader(pqfile, path_batch, batch_num)
train_dataset = ice_dataset
# proportions = [.75, .10, .15]
# lengths = [int(p * len(ice_dataset)) for p in proportions]
# lengths[-1] = len(ice_dataset) - sum(lengths[:-1])
# train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(ice_dataset, lengths)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn, num_workers=11)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn, num_workers=11)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn, num_workers=11)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 11 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
print(device)

cpu


In [ ]:
# model(ice_dataset.__getitem__(0)['input_tensor'].to(device))

In [ ]:
# print(ice_dataset.__getitem__(0)['input_tensor'].shape)
# for i in range(20):
#     print(ice_dataset.__getitem__(i)['input_tensor'].shape)

In [ ]:
# train_dataloader.__getitem__(0)
# # temp = iter(train_dataloader)
# # next(temp)

In [ ]:
params = {
    'num_epochs':num_epochs,
    'optimizer':optimizer,
    'loss_function':loss_function,
    'train_dataloader':train_dataloader,
    # 'val_dataloader': val_dataloader,
    # 'test_dataloader': test_dataloader,
    'device':device,
    'num_epoch' : num_epochs
}


In [ ]:
# train_dataset[0]['input_tensor'].shape

In [ ]:
# train_dataset[0]['label'] # This would be 'azimuth','zenith'

In [ ]:
import gc
train_losses = []

def train(model, params):
    model.train()
    loss_function=params["loss_function"]
    train_dataloader=params["train_dataloader"]
    # val_dataloader=params["val_dataloader"]
    # test_dataloader=params["test_dataloader"]

    # device=params["device"]
    # for epoch in range(0, num_epochs):
    #     with tqdm(train_dataloader,unit = 'batch') as tepoch:
    #         for dat in train_dataloader:
    #             tepoch.set_description(f"Epoch {epoch}")
    #             inputs, labels = dat['input_tensor'].to(device),dat['label'].to(device)
    #             optimizer.zero_grad()
    #             outputs = model(inputs).to(device)
    #             train_loss = loss_function(outputs.float(),labels.float())
    #             train_loss = train_loss.requires_grad_(True)
    #             train_loss.backward()
    #             optimizer.step()
    #             tepoch.set_postfix(loss=train_loss.item())

    aa = 0

    device=params["device"]
    for epoch in range(0, num_epochs):
        for dat in tqdm(train_dataloader):
            inputs, labels = dat['input_tensor'].to(device),dat['label'].to(device)
            optimizer.zero_grad()
            outputs = model(inputs).to(device)
            train_loss = loss_function(outputs.float(),labels.float())
            if aa%10 == 0:
                train_losses.append(train_loss.item())
            # if aa%100==0:
            #     print(f"Train loss: {train_loss.item()}")
            #     gc.collect()
            # train_loss = train_loss.requires_grad_(True)
            train_loss.backward()
            optimizer.step()
            # torch.cuda.empty_cache()
            aa = aa+1

    model.eval()
    # train_rmse = evaluation(train_dataloader)
    # val_rmse = evaluation(val_dataloader)

    # print(f"Train Loss: {train_rmse}")
    # print(" Train Loss: %.4f, Validation Loss: %.4f" %(train_rmse, val_rmse))
    # torch.cuda.empty_cache()
    # gc.collect()
    return 0


train(model, params)

  0%|          | 190/50000 [04:49<14:44:38,  1.07s/it]

In [13]:
from datetime import datetime

# Get current date and time
now = datetime.now()

# Format the date and time (e.g., "2025-06-07 15:30:45")
timestamp = now.strftime("%Y-%m-%d_%H:%M:%S")

In [14]:
name = "/content/drive/MyDrive/model_101_"+timestamp+".pth"
torch.save(model.state_dict(), name)

In [15]:
import pickle

# Save to file
name = "/content/drive/MyDrive/losses"+timestamp+".pickle"
with open(name, 'wb') as f:
    pickle.dump(train_losses, f)

NameError: name 'train_losses' is not defined

In [ ]:
# torch.save(model, "./model_all.pth")

In [ ]:
# pqfile = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/test_meta.parquet'
# path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/test/'
# batch_num=661
# inference_dataset = ICECUDE_Dataset(pqfile,path_batch,batch_num,'test')
# inference_dataloader = DataLoader(inference_dataset, batch_size=1)

In [ ]:
# model.eval()
# output_lst=[]
# for dat in inference_dataloader:
#     inputs = dat['input_tensor'].to(device)
#     outputs = model(inputs).to(device)
#     outputs = outputs.cpu().detach().squeeze().numpy()
#     outputs = outputs.tolist()
#     output_lst.append(outputs)

In [ ]:
# sample_sub = pq.read_table('/kaggle/input/icecube-neutrinos-in-deep-ice/sample_submission.parquet').to_pandas()
# batch661 = pq.read_table('/kaggle/input/icecube-neutrinos-in-deep-ice/test/batch_661.parquet').to_pandas()


In [ ]:
# sample_sub

In [ ]:
# submission = pd.DataFrame(output_lst)
# submission.index = batch661.index.unique().tolist()
# submission.reset_index(inplace = True)
# submission.columns = ['event_id','azimuth','zenith']

In [ ]:
# submission